In [1]:
import pandas as pd
import requests
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [2]:
hubs = ['ALBE', 'ALDE', 'AT', 'BE', 'CZ', 'DE_LU', 'FR', 'HR', 'HU', 'NL', 'PL', 'RO', 'SI', 'SK']

start = datetime(2022, 9, 1).strftime('%Y-%m-%d')
#end = datetime(2022, 9, 2).strftime('%Y-%m-%d')
end = datetime(2023, 3, 31).strftime('%Y-%m-%d')

In [3]:
def retrieve_ptdf(fromDate, toDate):
    data = {
        'fromUtc': f'{fromDate}T00:00:00.000Z',
        'toUtc': f'{toDate}T23:00:00.000Z',
    }

    try:
        response = requests.post('https://publicationtool.jao.eu/core/api/data/preFinalComputation', json=data)
        response.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
   
    return pd.DataFrame(response.json()['data'])

In [4]:
df_grid = pd.read_excel('../flow_based_domain/grid.xlsx', index_col=0)

In [5]:
df_ptdf_test = retrieve_ptdf(start, end)
display(df_ptdf_test.iloc[0]['contingencies'])

500 Server Error: Internal Server Error for url: https://publicationtool.jao.eu/core/api/data/preFinalComputation


KeyError: 'data'

In [6]:
# ZONAL PTDF and RAM
df_ptdf = retrieve_ptdf(start, end)

df_ptdf = df_ptdf.rename(columns={
    'dateTimeUtc': 'DateTime'
})
df_ptdf = df_ptdf.set_index('DateTime')
df_ptdf.index = pd.to_datetime(df_ptdf.index)
df_ptdf = df_ptdf.tz_localize(None)
df_ptdf = df_ptdf.reset_index()

df_ptdf = df_ptdf[df_ptdf.cneEic != 'NA']
df_ptdf = df_ptdf[[
    'DateTime', 'cneName', 'cneEic', 'tso', 'direction', 'substationFrom', 'substationTo', 'contingencies', 'ram', 'ptdf_ALBE', 'ptdf_ALDE', 'ptdf_AT', 'ptdf_BE', 'ptdf_CZ', 'ptdf_DE', 'ptdf_FR', 'ptdf_HR', 'ptdf_HU', 'ptdf_NL', 
    'ptdf_PL', 'ptdf_RO', 'ptdf_SI', 'ptdf_SK'
]]

display(df_ptdf)
#df_ptdf.to_excel("ptdf_z_raw.xlsx")

,DateTime,cneName,cneEic,tso,direction,substationFrom,substationTo,contingencies,ram,ptdf_ALBE,ptdf_ALDE,ptdf_AT,ptdf_BE,ptdf_CZ,ptdf_DE,ptdf_FR,ptdf_HR,ptdf_HU,ptdf_NL,ptdf_PL,ptdf_RO,ptdf_SI,ptdf_SK
0,2022-09-01 00:00:00,Ernsthofen 2 - Hausruck 203C,14T-220-0-0203CD,APG,DIRECT,Ernsthofen 2,Hausruck,"[{'number': 1, 'branchName': 'St. Peter 2 - St...",439.0,-0.06186,-0.06238,-0.06499,-0.06181,-0.02490,-0.06303,-0.06025,-0.00273,0.00340,-0.06144,-0.02183,0.00010,-0.00259,-0.00200
1,2022-09-01 00:00:00,Ernsthofen 2 - Hausruck 203C,14T-220-0-0203CD,APG,OPPOSITE,Ernsthofen 2,Hausruck,"[{'number': 1, 'branchName': 'St. Peter 2 - St...",341.0,0.06186,0.06238,0.06499,0.06181,0.02490,0.06303,0.06025,0.00273,-0.00340,0.06144,0.02183,-0.00010,0.00259,0.00200
2,2022-09-01 00:00:00,Wien Suedost - Gyoer 245,10T-AT-HU-00002U,APG,DIRECT,Wien Suedost,Gyoer 245,"[{'number': 1, 'branchName': 'Gyor - Neusiedl'...",146.0,0.02502,0.02502,0.03672,0.02504,0.02301,0.02519,0.02522,0.01195,-0.01617,0.02482,0.01430,-0.00085,0.02081,0.00497
3,2022-09-01 00:00:00,Neusiedl - Gyoer 246B,10T-AT-HU-00001W,APG,DIRECT,Neusiedl,Gyoer 246B,"[{'number': 1, 'branchName': 'Gyor - Wien', 'b...",155.0,0.02473,0.02472,0.03740,0.02476,0.02248,0.02487,0.02501,0.01253,-0.01655,0.02451,0.01386,-0.00089,0.02190,0.00451
4,2022-09-01 00:00:00,Neusiedl - Gyoer 246B,10T-AT-HU-00001W,APG,OPPOSITE,Neusiedl,Gyoer 246B,"[{'number': 1, 'branchName': 'Gyor - Wien', 'b...",267.0,-0.02473,-0.02472,-0.03740,-0.02476,-0.02248,-0.02487,-0.02501,-0.01253,0.01655,-0.02451,-0.01386,0.00089,-0.02190,-0.00451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541476,2023-03-31 22:00:00,Krizovany - Sokolnice,10T-CZ-SK-00003U,SEPS,DIRECT,Krizovany,Sokolnice,"[{'number': 1, 'branchName': 'Sokolnice - Stup...",1421.0,-0.12423,-0.12522,-0.10793,-0.12392,-0.18969,-0.12661,-0.12154,-0.03307,0.01876,-0.12507,-0.07989,0.00276,-0.06246,0.06681
541477,2023-03-31 22:00:00,Sokolnice - Stupava,10T-CZ-SK-00005Q,SEPS,OPPOSITE,Stupava,Sokolnice,"[{'number': 1, 'branchName': 'Krizovany - Soko...",1004.0,0.11961,0.12068,0.09994,0.11927,0.18129,0.12210,0.11665,0.02803,-0.02225,0.12060,0.08492,-0.00214,0.05525,-0.07919
541478,2023-03-31 22:00:00,V.Kapusany - Mukachevo (WPS),10T-SK-UA-00001W,SEPS,OPPOSITE,Velke Kapusany,Mukachevo,"[{'number': 1, 'branchName': 'R.Sobota - Sajoi...",1388.0,-0.21769,-0.21902,-0.19030,-0.21719,-0.21745,-0.22024,-0.21368,-0.11134,-0.09501,-0.21954,-0.29591,0.01699,-0.15569,-0.28346
541479,2023-03-31 22:00:00,Nosovice - Varin,10T-CZ-SK-00002W,SEPS,OPPOSITE,Nosovice,Varin,"[{'number': 1, 'branchName': 'Krizovany - Soko...",1179.0,0.15186,0.15387,0.09884,0.15112,0.15771,0.15581,0.14591,0.03331,-0.00911,0.15457,0.19936,-0.00316,0.05968,-0.06298


In [7]:
df_ptdf.columns = ['DateTime', 'name', 'eic', 'tso', 'direction', 'substation_from', 'substation_to', 'contingencies', 'ram'] + hubs
df_ptdf['contingencies'] = df_ptdf['contingencies'].apply(lambda c_list: list(filter(lambda c: c['elementType'] in ['Line', 'TieLine'], c_list)))

#df_ptdf_test = df_ptdf.copy()
#df_ptdf_test['contingencies_count'] = df_ptdf_test['contingencies'].apply(lambda c_list: len(c_list))
#display(df_ptdf_test[['ram', 'eic', 'tso', 'contingencies_count']].groupby(['eic', 'tso', 'contingencies_count']).count())
#display(df_ptdf_test)

In [8]:
# CONTINGENCY MAPPING

contingency_tsos = []
contingency_eics = []
contingency_names = []
contingency_obs_ids = []

for index, row in df_ptdf[df_ptdf.contingencies.map(len) > 0].iterrows():
    for cont in row['contingencies']:
        contingency_tsos.append(row['tso'])
        contingency_eics.append(cont['branchEic'])
        contingency_names.append(cont['branchName'])
        contingency_obs_ids.append(index)

In [9]:
df_contingencies = pd.DataFrame(
    [contingency_tsos, contingency_eics, contingency_names, contingency_obs_ids],
    index=['tso', 'eic', 'name', 'obs_id']
).T

df_contingencies_unique = df_contingencies.groupby(['tso', 'eic', 'name']).obs_id.apply(list).reset_index()

display(df_contingencies)
display(df_contingencies_unique)

,tso,eic,name,obs_id
0,APG,28T000000000301K,400 kV Cirkovce - Podlog,5
1,APG,28T000000000301K,400 kV Cirkovce - Podlog,6
2,APG,11TD2L000000201H,Pirach - Pleinting 257,7
3,APG,14T-380-0-0440AX,Sarasdorf - Zurndorf 440A,10
4,ELIA,22T-BE-IN-LI018R,Lixhe - Gramme 380.11,18
...,...,...,...,...
325504,SEPS,10T1001C--00054V,R.Sobota - Sajoivanka,541478
325505,SEPS,10T-CZ-SK-00003U,Krizovany - Sokolnice,541479
325506,SEPS,10T-CZ-SK-00003U,Sokolnice - Krizovany,541479
325507,SEPS,10T-CZ-SK-00005Q,Sokolnice - Stupava,541480


,tso,eic,name,obs_id
0,50HERTZ,10T-CZ-DE-00002U,Hradec - Rohrsdorf - V445,"[448484, 448590, 448695, 448799, 448903, 44920..."
1,50HERTZ,10T-CZ-DE-00002U,Hradec - Rohrsdorf 445,"[448484, 448590, 448695, 448799, 448903, 44920..."
2,50HERTZ,10T-CZ-DE-00003S,Hradec - Rohrsdorf - V446,"[448483, 448589, 448694, 448798, 448902, 44920..."
3,50HERTZ,10T-CZ-DE-00003S,Hradec - Rohrsdorf 446,"[448483, 448589, 448694, 448798, 448902, 44920..."
4,50HERTZ,10T-DE-PL-00001D,Hagenwerder - Mikulowa 568,"[450432, 450535, 450649, 450769, 450870, 45271..."
...,...,...,...,...
738,TRANSNETBW,12T-000000142I-U,Pradella-Sils LN301360,"[476943, 477058, 477174, 477280, 477386, 47749..."
739,TRANSNETBW,14T-220-0-0405CY,Meiningen - Werben 406C,"[37607, 38833, 38933, 39139]"
740,TRANSNETBW,14T-221-0-0405BQ,Walgau - Werben 405B,"[637, 872, 982, 1092, 1195, 1302, 1408, 1524, ..."
741,TRANSNETBW,17T-FR-000000317,L 400kV N0 1 MUHLBACH-SIERENTZ,"[5465, 12932, 13246, 13357, 13456, 19944, 2526..."


In [10]:
df_contingencies_unique = df_contingencies.groupby(['tso', 'eic', 'name']).obs_id.apply(list).reset_index()

for index, row in df_grid.iterrows():
    if row['tso'] == 'TENNET NL':
        df_grid.loc[index, 'tso'] = 'TENNETBV'
    if row['tso'] == 'Amprion GmbH':
        df_grid.loc[index, 'tso'] = 'AMPRION'
    if row['tso'] == 'TEL':
        df_grid.loc[index, 'tso'] = 'TRANSELECTRICA'
        

manual_tso = []
manual_eic = []
manual_name = []
for index, row in df_contingencies_unique.iterrows():
    if len(df_grid[(df_grid.tso == row['tso']) & (df_grid.eic == row['eic'])]) > 1:
        manual_tso.append(row['tso'])
        manual_eic.append(row['eic'])
        manual_name.append(row['name'])
    elif len(df_grid[(df_grid.tso == row['tso']) & (df_grid.eic == row['eic'])]) > 0:
        if len(df_grid[df_grid.eic == row['eic']]) > 1:
            if len(df_grid[(df_grid.eic == row['eic']) & (df_grid.name.str[:2].str.lower() == row['name'][:2].lower())]) > 0:
                df_contingencies_unique.loc[index, 'line_id'] = df_grid[(df_grid.eic == row['eic']) & (df_grid.name.str[:2].str.lower() == row['name'][:2].lower())].index[0]
            else:
                if len(df_grid[(df_grid.eic == row['eic']) & ~(df_grid.name.str[:2].str.lower() == row['name'].split('kV ')[-1][:2].lower())]) > 1:
                    print(row['tso'], row['eic'], row['name'].split('kV ')[-1])
                    if row['eic'] == '11T-D4-D7-00005O':
                        df_contingencies_unique.loc[index, 'line_id'] = 2589
                        print('Done')
                    if row['eic'] == '10T-HR-HU-000041':
                        df_contingencies_unique.loc[index, 'line_id'] = 2776 
                        print('Done')
                else:
                    df_contingencies_unique.loc[index, 'line_id'] = df_grid[(df_grid.eic == row['eic']) & ~(df_grid.name.str[:2].str.lower() == row['name'].split('kV ')[-1][:2].lower())].index[0]
        else:
            df_contingencies_unique.loc[index, 'line_id'] = df_grid[(df_grid.tso == row['tso']) & (df_grid.eic == row['eic'])].index[0]
        
df_manual_cont_selection = pd.DataFrame({
    "tso": manual_tso,
    "eic": manual_eic,
    "name": manual_name,
})

#df_manual_cont_selection.to_excel("manual_cont_selection.xlsx")

AMPRION 11T-D4-D7-00005O Voehringen - Dellmensingen DONAU W
Done
HOPS 10T-HR-HU-000041 Ernestinovo - Pecs 2
Done


In [11]:
df_manual_cont_selection_completed = pd.read_excel("manual_cont_selection_completed.xlsx", index_col=0)

for index, row in df_contingencies_unique.iterrows():
    if len(df_grid[(df_grid.tso == row['tso']) & (df_grid.eic == row['eic'])]) > 1:
        df_contingencies_unique.loc[index, 'line_id'] = df_manual_cont_selection_completed[
            (df_manual_cont_selection_completed.tso == row['tso']) &
            (df_manual_cont_selection_completed.eic == row['eic']) &
            (df_manual_cont_selection_completed.name == row['name'])
         ].iloc[0]['line_id']
        
df_contingencies_unique = df_contingencies_unique[~df_contingencies_unique.line_id.isna()]
df_contingencies_unique.line_id = df_contingencies_unique.line_id.astype(int)

df_ptdf['contingency_list'] = df_ptdf.contingencies.copy().apply(lambda x: []).astype(object)

for index, row in df_contingencies_unique.iterrows():
    for obs_id in row['obs_id']:
        df_ptdf.loc[obs_id, 'contingency_list'].append(row['line_id']) 
        
display(df_contingencies_unique[df_contingencies_unique.eic == '22T-BE-IN-LI019P'])

,tso,eic,name,obs_id,line_id
366,ELIA,22T-BE-IN-LI019P,Y-Courcelles (-Gramme - Tergnee) 380.31,"[4461, 5005, 6841, 7052, 7158, 7258, 7361, 746...",714
367,ELIA,22T-BE-IN-LI019P,Y-Gramme (-Courcelles - Tergnee) 380.31,"[4461, 5005, 6841, 7052, 7158, 7258, 7361, 746...",715
368,ELIA,22T-BE-IN-LI019P,Y-Tergnee (-Courcelles -Gramme) 380.31,"[4461, 5005, 6841, 7052, 7158, 7258, 7361, 746...",716


In [12]:
for index, row in df_ptdf[df_ptdf.contingencies.map(len) == 2].iterrows():
    cont = row['contingency_list']
    if len(cont) > 0:
        if cont[0] == cont[1]:
            lines = df_grid[df_grid.eic == df_grid[df_grid.index == cont[0]].iloc[0]['eic']].index.to_list()
            if len(lines) > 1:
                df_ptdf.loc[index, 'contingency_list'] = [[lines[0]], [lines[1]]]
            else:
                df_ptdf.loc[index, 'contingency_list'] = [[lines[0]]]

display(df_ptdf)

,DateTime,name,eic,tso,direction,substation_from,substation_to,contingencies,ram,ALBE,ALDE,AT,BE,CZ,DE_LU,FR,HR,HU,NL,PL,RO,SI,SK,contingency_list
0,2022-09-01 00:00:00,Ernsthofen 2 - Hausruck 203C,14T-220-0-0203CD,APG,DIRECT,Ernsthofen 2,Hausruck,[],439.0,-0.06186,-0.06238,-0.06499,-0.06181,-0.02490,-0.06303,-0.06025,-0.00273,0.00340,-0.06144,-0.02183,0.00010,-0.00259,-0.00200,[]
1,2022-09-01 00:00:00,Ernsthofen 2 - Hausruck 203C,14T-220-0-0203CD,APG,OPPOSITE,Ernsthofen 2,Hausruck,[],341.0,0.06186,0.06238,0.06499,0.06181,0.02490,0.06303,0.06025,0.00273,-0.00340,0.06144,0.02183,-0.00010,0.00259,0.00200,[]
2,2022-09-01 00:00:00,Wien Suedost - Gyoer 245,10T-AT-HU-00002U,APG,DIRECT,Wien Suedost,Gyoer 245,[],146.0,0.02502,0.02502,0.03672,0.02504,0.02301,0.02519,0.02522,0.01195,-0.01617,0.02482,0.01430,-0.00085,0.02081,0.00497,[]
3,2022-09-01 00:00:00,Neusiedl - Gyoer 246B,10T-AT-HU-00001W,APG,DIRECT,Neusiedl,Gyoer 246B,[],155.0,0.02473,0.02472,0.03740,0.02476,0.02248,0.02487,0.02501,0.01253,-0.01655,0.02451,0.01386,-0.00089,0.02190,0.00451,[]
4,2022-09-01 00:00:00,Neusiedl - Gyoer 246B,10T-AT-HU-00001W,APG,OPPOSITE,Neusiedl,Gyoer 246B,[],267.0,-0.02473,-0.02472,-0.03740,-0.02476,-0.02248,-0.02487,-0.02501,-0.01253,0.01655,-0.02451,-0.01386,0.00089,-0.02190,-0.00451,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541476,2023-03-31 22:00:00,Krizovany - Sokolnice,10T-CZ-SK-00003U,SEPS,DIRECT,Krizovany,Sokolnice,"[{'number': 1, 'branchName': 'Sokolnice - Stup...",1421.0,-0.12423,-0.12522,-0.10793,-0.12392,-0.18969,-0.12661,-0.12154,-0.03307,0.01876,-0.12507,-0.07989,0.00276,-0.06246,0.06681,"[2731, 2815]"
541477,2023-03-31 22:00:00,Sokolnice - Stupava,10T-CZ-SK-00005Q,SEPS,OPPOSITE,Stupava,Sokolnice,"[{'number': 1, 'branchName': 'Krizovany - Soko...",1004.0,0.11961,0.12068,0.09994,0.11927,0.18129,0.12210,0.11665,0.02803,-0.02225,0.12060,0.08492,-0.00214,0.05525,-0.07919,"[2814, 2722]"
541478,2023-03-31 22:00:00,V.Kapusany - Mukachevo (WPS),10T-SK-UA-00001W,SEPS,OPPOSITE,Velke Kapusany,Mukachevo,"[{'number': 1, 'branchName': 'R.Sobota - Sajoi...",1388.0,-0.21769,-0.21902,-0.19030,-0.21719,-0.21745,-0.22024,-0.21368,-0.11134,-0.09501,-0.21954,-0.29591,0.01699,-0.15569,-0.28346,"[2781, 2810]"
541479,2023-03-31 22:00:00,Nosovice - Varin,10T-CZ-SK-00002W,SEPS,OPPOSITE,Nosovice,Varin,"[{'number': 1, 'branchName': 'Krizovany - Soko...",1179.0,0.15186,0.15387,0.09884,0.15112,0.15771,0.15581,0.14591,0.03331,-0.00911,0.15457,0.19936,-0.00316,0.05968,-0.06298,"[2814, 2722]"


In [13]:
for index, row in df_ptdf[df_ptdf.contingencies.map(len) > 2].iterrows():
    #df_ptdf.loc[index, 'contingency_list'] = 
    if len(row['contingency_list']) > 2:
        if row['contingency_list'][0] == row['contingency_list'][1]:
            df_ptdf.loc[index, 'contingency_list'] = [[row['contingency_list'][1]], [row['contingency_list'][2]], [row['contingency_list'][3]]]

In [17]:
df_ptdf['contingency_list'] = df_ptdf['contingency_list'].apply(sorted)
display(df_ptdf)

,DateTime,name,eic,tso,direction,substation_from,substation_to,contingencies,ram,ALBE,ALDE,AT,BE,CZ,DE_LU,FR,HR,HU,NL,PL,RO,SI,SK,contingency_list,contingency_list2
0,2022-09-01 00:00:00,Ernsthofen 2 - Hausruck 203C,14T-220-0-0203CD,APG,DIRECT,Ernsthofen 2,Hausruck,[],439.0,-0.06186,-0.06238,-0.06499,-0.06181,-0.02490,-0.06303,-0.06025,-0.00273,0.00340,-0.06144,-0.02183,0.00010,-0.00259,-0.00200,[],[]
1,2022-09-01 00:00:00,Ernsthofen 2 - Hausruck 203C,14T-220-0-0203CD,APG,OPPOSITE,Ernsthofen 2,Hausruck,[],341.0,0.06186,0.06238,0.06499,0.06181,0.02490,0.06303,0.06025,0.00273,-0.00340,0.06144,0.02183,-0.00010,0.00259,0.00200,[],[]
2,2022-09-01 00:00:00,Wien Suedost - Gyoer 245,10T-AT-HU-00002U,APG,DIRECT,Wien Suedost,Gyoer 245,[],146.0,0.02502,0.02502,0.03672,0.02504,0.02301,0.02519,0.02522,0.01195,-0.01617,0.02482,0.01430,-0.00085,0.02081,0.00497,[],[]
3,2022-09-01 00:00:00,Neusiedl - Gyoer 246B,10T-AT-HU-00001W,APG,DIRECT,Neusiedl,Gyoer 246B,[],155.0,0.02473,0.02472,0.03740,0.02476,0.02248,0.02487,0.02501,0.01253,-0.01655,0.02451,0.01386,-0.00089,0.02190,0.00451,[],[]
4,2022-09-01 00:00:00,Neusiedl - Gyoer 246B,10T-AT-HU-00001W,APG,OPPOSITE,Neusiedl,Gyoer 246B,[],267.0,-0.02473,-0.02472,-0.03740,-0.02476,-0.02248,-0.02487,-0.02501,-0.01253,0.01655,-0.02451,-0.01386,0.00089,-0.02190,-0.00451,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541476,2023-03-31 22:00:00,Krizovany - Sokolnice,10T-CZ-SK-00003U,SEPS,DIRECT,Krizovany,Sokolnice,"[{'number': 1, 'branchName': 'Sokolnice - Stup...",1421.0,-0.12423,-0.12522,-0.10793,-0.12392,-0.18969,-0.12661,-0.12154,-0.03307,0.01876,-0.12507,-0.07989,0.00276,-0.06246,0.06681,"[2731, 2815]","[2731, 2815]"
541477,2023-03-31 22:00:00,Sokolnice - Stupava,10T-CZ-SK-00005Q,SEPS,OPPOSITE,Stupava,Sokolnice,"[{'number': 1, 'branchName': 'Krizovany - Soko...",1004.0,0.11961,0.12068,0.09994,0.11927,0.18129,0.12210,0.11665,0.02803,-0.02225,0.12060,0.08492,-0.00214,0.05525,-0.07919,"[2722, 2814]","[2722, 2814]"
541478,2023-03-31 22:00:00,V.Kapusany - Mukachevo (WPS),10T-SK-UA-00001W,SEPS,OPPOSITE,Velke Kapusany,Mukachevo,"[{'number': 1, 'branchName': 'R.Sobota - Sajoi...",1388.0,-0.21769,-0.21902,-0.19030,-0.21719,-0.21745,-0.22024,-0.21368,-0.11134,-0.09501,-0.21954,-0.29591,0.01699,-0.15569,-0.28346,"[2781, 2810]","[2781, 2810]"
541479,2023-03-31 22:00:00,Nosovice - Varin,10T-CZ-SK-00002W,SEPS,OPPOSITE,Nosovice,Varin,"[{'number': 1, 'branchName': 'Krizovany - Soko...",1179.0,0.15186,0.15387,0.09884,0.15112,0.15771,0.15581,0.14591,0.03331,-0.00911,0.15457,0.19936,-0.00316,0.05968,-0.06298,"[2722, 2814]","[2722, 2814]"


In [52]:
df_ptdf = df_ptdf[df_ptdf.direction != 'NA']

all_eics = df_ptdf.eic.unique()
unique_eics = []
invalid_obs_eics = []
non_unique_eics = []

for eic in all_eics:
    num_cne = len(df_grid[df_grid.eic == eic])
    if num_cne == 0:
        invalid_obs_eics.append(eic)
    elif num_cne == 1:
        unique_eics.append(eic)
    else:
        non_unique_eics.append(eic)
        
print('Total:', len(all_eics))
print('Unique:', len(unique_eics))
print('Non-valid observed eics:', len(invalid_obs_eics))

df_ptdf_adj = df_ptdf[~df_ptdf.eic.isin(invalid_obs_eics)]
df_ptdf_adj['line_id'] = None

for index, row in df_grid.iterrows():
    if row['tso'] == 'TENNET NL':
        df_grid.loc[index, 'tso'] = 'TENNETBV'
    if row['tso'] == 'Amprion GmbH':
        df_grid.loc[index, 'tso'] = 'AMPRION'
    if row['tso'] == 'TEL':
        df_grid.loc[index, 'tso'] = 'TRANSELECTRICA'

for eic in unique_eics:
    line_name = df_grid[df_grid.eic == eic].iloc[0]['name']
    print(line_name)
    line_id = df_grid[df_grid.eic == eic].index[0]
    df_ptdf_adj.loc[df_ptdf_adj.eic == eic, 'line_id'] = line_id
    df_ptdf_adj.loc[
        (df_ptdf_adj.eic == eic) &
        (line_name[:2] != df_ptdf_adj.name.str[:2])
    , hubs] = -1 * df_ptdf_adj.loc[
        (df_ptdf_adj.eic == eic) &
        (line_name[:2] != df_ptdf_adj.name.str[:2])
    , hubs]

df_ptdf_non_unique_eic = df_ptdf[df_ptdf.eic.isin(non_unique_eics)].groupby(['eic', 'direction', 'tso']).head(1).sort_values(by=['eic'])

identified_pair_eic = []

identified_tielines_eic = []
tielines = []
remainder_eics = []

for eic in non_unique_eics:
    df_ptdf_nu_eic = df_ptdf_non_unique_eic[df_ptdf_non_unique_eic.eic == eic]
    
    for tso in df_ptdf_nu_eic.tso.unique():
        df_pair = df_ptdf_nu_eic[(df_ptdf_nu_eic.tso == tso)]
        df_pair_direct = df_pair[df_pair.direction == 'DIRECT']
        df_pair_opposite = df_pair[df_pair.direction == 'OPPOSITE']
        
        df_line = df_grid[(df_grid.eic == eic) & (df_grid.tso == tso)]
        if len(df_line) == 2: # duplicate because of two directions
            identified_pair_eic.append(eic)
            if df_line.iloc[0]['name'][:2] == df_pair_direct.iloc[0]['name'][:2]:
                df_ptdf_adj.loc[
                    (df_ptdf_adj.eic == eic) & 
                    (df_ptdf_adj.tso == tso) & 
                    (df_ptdf_adj.direction == 'DIRECT'), 'line_id'] = df_line.index[0]
            else:
                df_ptdf_adj.loc[
                    (df_ptdf_adj.eic == eic) & 
                    (df_ptdf_adj.tso == tso) & 
                    (df_ptdf_adj.direction == 'OPPOSITE'), 'line_id'] = df_line.index[0] 

            if df_line.iloc[1]['name'][:2] == df_pair_direct.iloc[0]['name'][:2]:
                df_ptdf_adj.loc[
                    (df_ptdf_adj.eic == eic) & 
                    (df_ptdf_adj.tso == tso) & 
                    (df_ptdf_adj.direction == 'DIRECT'), 'line_id'] = df_line.index[1]
            else:
                df_ptdf_adj.loc[
                    (df_ptdf_adj.eic == eic) & 
                    (df_ptdf_adj.tso == tso) & 
                    (df_ptdf_adj.direction == 'OPPOSITE'), 'line_id'] = df_line.index[1] 
        
        elif len(df_line) == 1: # it was duplicate because of two TSOs
            identified_tielines_eic.append(eic)
            df_ptdf_adj.loc[(df_ptdf_adj.eic == eic) & (df_ptdf_adj.tso == tso), 'line_id'] = df_line.index[0]

        else:
            remainder_eics.append(eic)
            
print('Identified pair EICs: ', len(identified_pair_eic))
print('Identified tieline EICs: ', len(identified_tielines_eic))
df_ptdf_non_unique_eic = df_ptdf_non_unique_eic[df_ptdf_non_unique_eic.eic.isin(remainder_eics)]

display(df_ptdf_non_unique_eic)

#for index, eic in enumerate(remainder_eic):
#    df_ptdf_adj.loc[(df_ptdf_adj.eic == eic) & (df_ptdf_adj.tso == remainder_tso[index]) & (df_ptdf_adj.direction == remainder_direction[index]), 'line_id'] = remainder_line_ids[index]
    
display(df_ptdf_adj[df_ptdf_adj.line_id.isna()])

Total: 372
Unique: 202
Non-valid observed eics: 77


C:\Users\balazs.riskutia\AppData\Local\Temp\ipykernel_576\3132903498.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ptdf_adj['line_id'] = None


Ernsthofen - Hausruck 203C
Sarasdorf - Zurndorf 439A
ACHENE – GRAMME 380.10
GRAMME – LIXHE 380.11
LIXHE – VAN EYCK 380.91
Y-Simbach (-St. Peter - Altheim) 233/230
Mittelbexbach - Uchtelfangen BLIES S 
Schmölln - Dresden/Süd 555
Vierraden - Pasewalk 306
Gonyu - Gyor
Gyor - Oroszlany
Felsozsolca - Sajoivanka
RZESZOW-KROSNO ISKRZYNIA
Paroseni - Targiu Jiu Nord
Gabcikovo - P. Biskupice
H. Zdana - Sucany
V. Dur - Krizovany
V. Dur - Levice
Y-Simbach (-St. Peter - Altheim) 234/230
Paffendorf - Rommerskirchen PAFFEN S 
St.Peter - Altheim 233/230
Westtirol - Silz 414
Y-Oberzier (-Paffendorf - Sechtem) SECHTM S 
Liter - Martonvasar
Krimpen a/d IJssel-Geertruidenberg 380 W
V. Dur - Levice
Resita - Timisoara c1
Portile de Fier - Resita c1
Pulgar - Vieselbach 589
Dunamenti - Oroszlany
Borssele-Rilland 380 G
Diemen-Lelystad 380 W
Krimpen a/d IJssel-Breukelen Kortrijk 380 W
Oostzaan-Diemen 380 G
Hessenberg - Weissenbach 223
Altheim - Sittling 220
Lelystad-Ens 380 W
Ens-Zwolle 380 W
LIT 400kV N0 1 AVE

,DateTime,name,eic,tso,direction,substation_from,substation_to,contingencies,ram,ALBE,ALDE,AT,BE,CZ,DE_LU,FR,HR,HU,NL,PL,RO,SI,SK,contingency_list,contingency_list2
218013,2022-11-24 08:00:00,Y-Mercator (-Doel - Lillo) 380.51,22T-BE-IN-LI014Z,ELIA,OPPOSITE,Mercator,Liefkenshoek,"[{'number': 1, 'branchName': 'Doel - Zandvliet...",1015.0,-0.07687,0.04123,-0.00579,-0.17680,0.00750,0.02260,-0.08939,-0.00402,0.00144,0.16344,0.00968,0.00047,-0.00740,0.00379,[656],[656]
373991,2023-01-24 03:00:00,Y-Doel (-Lillo - Mercator) 380.52,22T-BE-IN-LI015X,ELIA,DIRECT,Doel,Liefkenshoek,"[{'number': 1, 'branchName': 'Doel - Zandvliet...",951.0,-0.08411,0.02233,-0.01350,-0.16905,-0.00192,0.00997,-0.08757,-0.01150,-0.00705,0.17886,0.00027,-0.00785,-0.01418,-0.00489,[656],[656]
626,2022-09-01 05:00:00,Y-Mercator (-Doel - Lillo) 380.52,22T-BE-IN-LI015X,ELIA,OPPOSITE,Mercator,Kallo,"[{'number': 1, 'branchName': 'Doel - Mercator ...",651.0,-0.05140,0.01479,-0.00286,-0.14117,0.00384,0.00921,-0.03562,-0.00177,0.00052,0.06122,0.00452,0.00016,-0.00307,0.00164,[657],[657]
39026,2022-09-16 12:00:00,Y-Gramme (-Courcelles - Tergnee) 380.31,22T-BE-IN-LI019P,ELIA,DIRECT,Gramme,Saint Amand,[],308.0,0.08956,-0.00906,0.00203,-0.04200,-0.00146,-0.00447,0.02067,0.00157,0.00010,-0.03743,-0.00205,0.00026,0.00226,-0.00050,[],[]
217694,2022-11-24 05:00:00,Y-Mercator (-Horta - Rodenhuize) 380.73,22T20161020---2C,ELIA,OPPOSITE,Mercator,Rodenhuize,"[{'number': 1, 'branchName': 'Y-Mercator (-Hor...",1525.0,-0.08971,-0.05940,0.00608,0.02369,-0.00952,-0.02574,0.12815,0.00480,-0.00169,-0.12351,-0.01183,-0.00037,0.00807,-0.00472,"[667, 682, 701]","[667, 682, 701]"
210750,2022-11-21 14:00:00,Y-Horta (-Mercator - Rodenhuize) 380.73,22T20161020---2C,ELIA,DIRECT,Horta,Rodenhuize,"[{'number': 1, 'branchName': 'Y-Mercator (-Hor...",1428.0,-0.08927,-0.05794,0.00144,-0.04404,-0.00920,-0.02621,0.11675,0.00439,-0.00164,-0.11337,-0.01120,-0.00041,0.00760,-0.00474,"[667, 682, 701]","[667, 682, 701]"
198065,2022-11-16 20:00:00,Y-Mercator (-Horta - Rodenhuize) 380.74,22T20161020---3A,ELIA,OPPOSITE,Mercator,Rodenhuize,"[{'number': 1, 'branchName': 'Y-Mercator (-Hor...",1512.0,-0.08414,-0.05675,0.00644,0.02745,-0.00991,-0.02016,0.12591,0.00469,-0.00183,-0.11558,-0.01201,-0.00055,0.00797,-0.00499,"[725, 726, 727]","[725, 726, 727]"
429540,2023-02-15 05:00:00,Y-Mercator (-Horta - Rodenhuize) 380.74,22T20161020---3A,ELIA,DIRECT,Mercator,Rodenhuize,[],3.0,0.15806,0.07222,-0.00759,0.03349,0.01144,0.03014,-0.14756,-0.00625,0.00214,0.13855,0.01385,0.00043,-0.01006,0.00560,[],[]
267,2022-09-01 02:00:00,Y-Van Eyck (-Andre Dumont - Gramme) 380.12,22T20161020---48,ELIA,OPPOSITE,Van Eyck,Zuttendaal,"[{'number': 1, 'branchName': 'Lixhe - Gramme 3...",1004.0,-0.09548,-0.03370,0.00127,0.20473,-0.00572,-0.01158,0.04929,0.00286,-0.00100,-0.03735,-0.00645,-0.00023,0.00357,-0.00264,[711],[711]
18,2022-09-01 00:00:00,Y-Gramme (-Andre Dumont - Van Eyck) 380.12,22T20161020---48,ELIA,DIRECT,Gramme,Zuttendaal,"[{'number': 1, 'branchName': 'Lixhe - Gramme 3...",995.0,-0.11957,-0.03186,0.00123,0.19074,-0.00547,-0.01111,0.04717,0.00272,-0.00096,-0.03748,-0.00620,-0.00022,0.00341,-0.00254,[711],[711]


,DateTime,name,eic,tso,direction,substation_from,substation_to,contingencies,ram,ALBE,ALDE,AT,BE,CZ,DE_LU,FR,HR,HU,NL,PL,RO,SI,SK,contingency_list,contingency_list2,line_id
18,2022-09-01 00:00:00,Y-Gramme (-Andre Dumont - Van Eyck) 380.12,22T20161020---48,ELIA,DIRECT,Gramme,Zuttendaal,"[{'number': 1, 'branchName': 'Lixhe - Gramme 3...",995.0,-0.11957,-0.03186,0.00123,0.19074,-0.00547,-0.01111,0.04717,0.00272,-0.00096,-0.03748,-0.00620,-0.00022,0.00341,-0.00254,[711],[711],None
139,2022-09-01 01:00:00,Y-Gramme (-Andre Dumont - Van Eyck) 380.12,22T20161020---48,ELIA,DIRECT,Gramme,Zuttendaal,"[{'number': 1, 'branchName': 'Lixhe - Gramme 3...",989.0,-0.11878,-0.03211,0.00125,0.19328,-0.00556,-0.01130,0.04783,0.00277,-0.00097,-0.03926,-0.00630,-0.00022,0.00347,-0.00259,[711],[711],None
267,2022-09-01 02:00:00,Y-Van Eyck (-Andre Dumont - Gramme) 380.12,22T20161020---48,ELIA,OPPOSITE,Van Eyck,Zuttendaal,"[{'number': 1, 'branchName': 'Lixhe - Gramme 3...",1004.0,-0.09548,-0.03370,0.00127,0.20473,-0.00572,-0.01158,0.04929,0.00286,-0.00100,-0.03735,-0.00645,-0.00023,0.00357,-0.00264,[711],[711],None
384,2022-09-01 03:00:00,Y-Van Eyck (-Andre Dumont - Gramme) 380.12,22T20161020---48,ELIA,OPPOSITE,Van Eyck,Zuttendaal,"[{'number': 1, 'branchName': 'Lixhe - Gramme 3...",1012.0,-0.09549,-0.03371,0.00127,0.20472,-0.00572,-0.01160,0.04914,0.00285,-0.00100,-0.03741,-0.00643,-0.00023,0.00356,-0.00251,[711],[711],None
508,2022-09-01 04:00:00,Y-Van Eyck (-Andre Dumont - Gramme) 380.12,22T20161020---48,ELIA,OPPOSITE,Van Eyck,Zuttendaal,"[{'number': 1, 'branchName': 'Lixhe - Gramme 3...",1077.0,-0.09544,-0.03366,0.00128,0.20477,-0.00572,-0.01267,0.04945,0.00282,-0.00098,-0.03934,-0.00638,-0.00024,0.00358,-0.00251,[711],[711],None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534679,2023-03-29 04:00:00,Y-Gramme (-Andre Dumont - Van Eyck) 380.12,22T20161020---48,ELIA,DIRECT,Gramme,Zuttendaal,"[{'number': 1, 'branchName': 'Lixhe - Gramme 3...",1035.0,-0.05135,-0.04097,0.00298,0.15441,-0.00502,-0.01543,0.05996,0.00252,-0.00101,-0.04018,-0.00831,-0.00037,0.00468,-0.00266,[711],[711],None
534898,2023-03-29 06:00:00,Y-Gramme (-Andre Dumont - Van Eyck) 380.12,22T20161020---48,ELIA,DIRECT,Gramme,Zuttendaal,"[{'number': 1, 'branchName': 'Lixhe - Gramme 3...",1065.0,-0.05337,-0.04449,0.00368,0.15331,-0.00549,-0.01734,0.06111,0.00315,-0.00127,-0.03530,-0.00885,-0.00044,0.00523,-0.00302,[711],[711],None
534899,2023-03-29 06:00:00,Y-Gramme (-Andre Dumont - Van Eyck) 380.12,22T20161020---48,ELIA,DIRECT,Gramme,Zuttendaal,"[{'number': 1, 'branchName': 'Lixhe - Van Eyck...",1085.0,0.22871,-0.04848,0.00402,0.17090,-0.00597,-0.01888,0.06619,0.00342,-0.00138,-0.03848,-0.00963,-0.00048,0.00569,-0.00329,[712],[712],None
535098,2023-03-29 08:00:00,Y-Gramme (-Andre Dumont - Van Eyck) 380.12,22T20161020---48,ELIA,DIRECT,Gramme,Zuttendaal,"[{'number': 1, 'branchName': 'Lixhe - Gramme 3...",1078.0,-0.05211,-0.04490,0.00424,0.15821,-0.00555,-0.01760,0.06239,0.00336,-0.00130,-0.03684,-0.00894,-0.00045,0.00531,-0.00303,[711],[711],None


In [12]:
remainder_eic = [
    '22T-BE-IN-LI014Z', 
    '22T-BE-IN-LI015X', 
    '22T-BE-IN-LI019P',
    '22T20161020---2C',
    '22T20161020---3A', 
    '22T20161020---48',
]

#df_ptdf_remainder_combinations = df_ptdf[df_ptdf.eic.isin(remainder_eic)].groupby(['eic', 'direction', 'tso', 'substation_from', 'substation_to']).head(1).sort_values(by=['eic'])
#df_ptdf_remainder_combinations[['name', 'eic', 'tso', 'substation_from', 'substation_to']].to_excel('eic_manual_completion.xlsx')

In [53]:
df_ptdf_remainder_filled = pd.read_excel('eic_manual_completion.xlsx', index_col=0)

for index, row in df_ptdf_remainder_filled.iterrows():
    df_ptdf_adj.loc[
        (df_ptdf_adj.eic == row['eic']) & 
        (df_ptdf_adj.tso == row['tso']) & 
        (df_ptdf_adj.substation_from == row['substation_from']) &
        (df_ptdf_adj.substation_to == row['substation_to']), 'line_id'] = row['line_id']
    
df_ptdf_adj['direction'] = df_ptdf_adj['direction'].apply(lambda x: (1 if x == 'DIRECT' else -1))
df_ptdf_adj = df_ptdf_adj[['DateTime', 'line_id', 'direction', 'contingency_list', 'ram'] + hubs]

display(df_ptdf_adj)
#df_ptdf_adj.to_excel("ptdf_z.xlsx")

C:\Users\balazs.riskutia\AppData\Local\Temp\ipykernel_576\1576530540.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ptdf_adj['direction'] = df_ptdf_adj['direction'].apply(lambda x: (1 if x == 'DIRECT' else -1))


,DateTime,line_id,direction,contingency_list,ram,ALBE,ALDE,AT,BE,CZ,DE_LU,FR,HR,HU,NL,PL,RO,SI,SK
0,2022-09-01 00:00:00,493,1,[],439.0,-0.06186,-0.06238,-0.06499,-0.06181,-0.02490,-0.06303,-0.06025,-0.00273,0.00340,-0.06144,-0.02183,0.00010,-0.00259,-0.00200
1,2022-09-01 00:00:00,493,-1,[],341.0,0.06186,0.06238,0.06499,0.06181,0.02490,0.06303,0.06025,0.00273,-0.00340,0.06144,0.02183,-0.00010,0.00259,0.00200
2,2022-09-01 00:00:00,2710,1,[],146.0,0.02502,0.02502,0.03672,0.02504,0.02301,0.02519,0.02522,0.01195,-0.01617,0.02482,0.01430,-0.00085,0.02081,0.00497
3,2022-09-01 00:00:00,2699,1,[],155.0,0.02473,0.02472,0.03740,0.02476,0.02248,0.02487,0.02501,0.01253,-0.01655,0.02451,0.01386,-0.00089,0.02190,0.00451
4,2022-09-01 00:00:00,2699,-1,[],267.0,-0.02473,-0.02472,-0.03740,-0.02476,-0.02248,-0.02487,-0.02501,-0.01253,0.01655,-0.02451,-0.01386,0.00089,-0.02190,-0.00451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541475,2023-03-31 22:00:00,2036,1,[2039],410.0,0.16091,0.16134,0.16359,0.16079,0.18497,0.16210,0.15977,0.08526,0.03712,0.16120,0.12676,-0.00840,0.13222,0.12419
541476,2023-03-31 22:00:00,2814,1,"[2731, 2815]",1421.0,-0.12423,-0.12522,-0.10793,-0.12392,-0.18969,-0.12661,-0.12154,-0.03307,0.01876,-0.12507,-0.07989,0.00276,-0.06246,0.06681
541477,2023-03-31 22:00:00,2815,-1,"[2722, 2814]",1004.0,0.11961,0.12068,0.09994,0.11927,0.18129,0.12210,0.11665,0.02803,-0.02225,0.12060,0.08492,-0.00214,0.05525,-0.07919
541479,2023-03-31 22:00:00,2813,-1,"[2722, 2814]",1179.0,0.15186,0.15387,0.09884,0.15112,0.15771,0.15581,0.14591,0.03331,-0.00911,0.15457,0.19936,-0.00316,0.05968,-0.06298


In [54]:
df_ptdf_adj['contingencies_string'] = df_ptdf_adj.contingency_list.apply(lambda s: ','.join([str(elem) for elem in s]))
df_observed_contingencies = pd.DataFrame({
    "lines": [[]] + list(df_ptdf_adj[df_ptdf_adj.contingency_list.map(len) > 0].groupby('contingencies_string').head(1).contingency_list)
})

display(df_observed_contingencies)

df_observed_contingencies.to_excel('contingencies.xlsx')

,lines
0,[]
1,[539]
2,[711]
3,"[698, 699, 702]"
4,[577]
...,...
325,[2160]
326,"[2740, 2768]"
327,"[2703, 2834]"
328,[306]


In [55]:
df_observed_contingencies['lines_string'] = df_observed_contingencies.lines.apply(lambda s: ','.join([str(elem) for elem in s]))

for index, row in df_ptdf_adj.iterrows():
    if len(row['contingency_list']) > 0:
        df_ptdf_adj.loc[index, 'contingency'] = df_observed_contingencies[
            df_observed_contingencies.lines_string == row['contingencies_string']
        ].index[0]
    else:
        df_ptdf_adj.loc[index, 'contingency'] = -1
        
df_ptdf_adj['contingency'] = df_ptdf_adj['contingency'].astype(int)

display(df_ptdf_adj)

,DateTime,line_id,direction,contingency_list,ram,ALBE,ALDE,AT,BE,CZ,DE_LU,FR,HR,HU,NL,PL,RO,SI,SK,contingencies_string,contingency
0,2022-09-01 00:00:00,493,1,[],439.0,-0.06186,-0.06238,-0.06499,-0.06181,-0.02490,-0.06303,-0.06025,-0.00273,0.00340,-0.06144,-0.02183,0.00010,-0.00259,-0.00200,,-1
1,2022-09-01 00:00:00,493,-1,[],341.0,0.06186,0.06238,0.06499,0.06181,0.02490,0.06303,0.06025,0.00273,-0.00340,0.06144,0.02183,-0.00010,0.00259,0.00200,,-1
2,2022-09-01 00:00:00,2710,1,[],146.0,0.02502,0.02502,0.03672,0.02504,0.02301,0.02519,0.02522,0.01195,-0.01617,0.02482,0.01430,-0.00085,0.02081,0.00497,,-1
3,2022-09-01 00:00:00,2699,1,[],155.0,0.02473,0.02472,0.03740,0.02476,0.02248,0.02487,0.02501,0.01253,-0.01655,0.02451,0.01386,-0.00089,0.02190,0.00451,,-1
4,2022-09-01 00:00:00,2699,-1,[],267.0,-0.02473,-0.02472,-0.03740,-0.02476,-0.02248,-0.02487,-0.02501,-0.01253,0.01655,-0.02451,-0.01386,0.00089,-0.02190,-0.00451,,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541475,2023-03-31 22:00:00,2036,1,[2039],410.0,0.16091,0.16134,0.16359,0.16079,0.18497,0.16210,0.15977,0.08526,0.03712,0.16120,0.12676,-0.00840,0.13222,0.12419,2039,27
541476,2023-03-31 22:00:00,2814,1,"[2731, 2815]",1421.0,-0.12423,-0.12522,-0.10793,-0.12392,-0.18969,-0.12661,-0.12154,-0.03307,0.01876,-0.12507,-0.07989,0.00276,-0.06246,0.06681,"2731,2815",289
541477,2023-03-31 22:00:00,2815,-1,"[2722, 2814]",1004.0,0.11961,0.12068,0.09994,0.11927,0.18129,0.12210,0.11665,0.02803,-0.02225,0.12060,0.08492,-0.00214,0.05525,-0.07919,"2722,2814",288
541479,2023-03-31 22:00:00,2813,-1,"[2722, 2814]",1179.0,0.15186,0.15387,0.09884,0.15112,0.15771,0.15581,0.14591,0.03331,-0.00911,0.15457,0.19936,-0.00316,0.05968,-0.06298,"2722,2814",288


In [57]:
df_ptdf_adj.loc[df_ptdf_adj.contingency.isna(), 'contingency'] = 0
df_ptdf_adj['contingency'] = df_ptdf_adj['contingency'].astype(int)
display(df_ptdf_adj)

,DateTime,line_id,direction,contingency_list,ram,ALBE,ALDE,AT,BE,CZ,DE_LU,FR,HR,HU,NL,PL,RO,SI,SK,contingencies_string,contingency
0,2022-09-01 00:00:00,493,1,[],439.0,-0.06186,-0.06238,-0.06499,-0.06181,-0.02490,-0.06303,-0.06025,-0.00273,0.00340,-0.06144,-0.02183,0.00010,-0.00259,-0.00200,,0
1,2022-09-01 00:00:00,493,-1,[],341.0,0.06186,0.06238,0.06499,0.06181,0.02490,0.06303,0.06025,0.00273,-0.00340,0.06144,0.02183,-0.00010,0.00259,0.00200,,0
2,2022-09-01 00:00:00,2710,1,[],146.0,0.02502,0.02502,0.03672,0.02504,0.02301,0.02519,0.02522,0.01195,-0.01617,0.02482,0.01430,-0.00085,0.02081,0.00497,,0
3,2022-09-01 00:00:00,2699,1,[],155.0,0.02473,0.02472,0.03740,0.02476,0.02248,0.02487,0.02501,0.01253,-0.01655,0.02451,0.01386,-0.00089,0.02190,0.00451,,0
4,2022-09-01 00:00:00,2699,-1,[],267.0,-0.02473,-0.02472,-0.03740,-0.02476,-0.02248,-0.02487,-0.02501,-0.01253,0.01655,-0.02451,-0.01386,0.00089,-0.02190,-0.00451,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541475,2023-03-31 22:00:00,2036,1,[2039],410.0,0.16091,0.16134,0.16359,0.16079,0.18497,0.16210,0.15977,0.08526,0.03712,0.16120,0.12676,-0.00840,0.13222,0.12419,2039,27
541476,2023-03-31 22:00:00,2814,1,"[2731, 2815]",1421.0,-0.12423,-0.12522,-0.10793,-0.12392,-0.18969,-0.12661,-0.12154,-0.03307,0.01876,-0.12507,-0.07989,0.00276,-0.06246,0.06681,"2731,2815",289
541477,2023-03-31 22:00:00,2815,-1,"[2722, 2814]",1004.0,0.11961,0.12068,0.09994,0.11927,0.18129,0.12210,0.11665,0.02803,-0.02225,0.12060,0.08492,-0.00214,0.05525,-0.07919,"2722,2814",288
541479,2023-03-31 22:00:00,2813,-1,"[2722, 2814]",1179.0,0.15186,0.15387,0.09884,0.15112,0.15771,0.15581,0.14591,0.03331,-0.00911,0.15457,0.19936,-0.00316,0.05968,-0.06298,"2722,2814",288


In [58]:
df_ptdf_adj_reduced = df_ptdf_adj.copy()
df_ptdf_adj_reduced.loc[df_ptdf_adj_reduced.direction == -1, hubs] = -1 * df_ptdf_adj_reduced.loc[df_ptdf_adj_reduced.direction == -1, hubs]


df_ptdf_adj_reduced = df_ptdf_adj_reduced[['DateTime', 'line_id', 'contingency', 'ram'] + hubs].groupby(['DateTime', 'line_id', 'contingency'], as_index=False).mean()
display(df_ptdf_adj_reduced)

,DateTime,line_id,contingency,ram,ALBE,ALDE,AT,BE,CZ,DE_LU,FR,HR,HU,NL,PL,RO,SI,SK
0,2022-09-01 00:00:00,12,0,153.0,-0.04836,-0.04861,-0.01764,-0.04791,-0.01287,-0.04590,-0.04244,-0.00756,0.00085,-0.05174,0.04220,0.00071,-0.01033,0.00715
1,2022-09-01 00:00:00,94,9,653.0,-0.01960,-0.02016,-0.02146,-0.01994,-0.07237,-0.03774,-0.02340,-0.00799,-0.00063,-0.01585,0.08202,0.00082,-0.01122,0.00769
2,2022-09-01 00:00:00,152,8,1140.0,-0.12361,-0.09008,0.00777,-0.13408,0.00672,-0.00222,-0.14070,-0.00395,0.00176,-0.07122,0.00364,0.00030,-0.00444,0.00304
3,2022-09-01 00:00:00,223,7,919.0,-0.08881,-0.11644,0.01279,-0.08286,-0.02146,-0.00737,0.00638,0.00944,-0.00309,-0.09761,-0.02508,-0.00078,0.01233,-0.00938
4,2022-09-01 00:00:00,333,6,887.0,0.08342,0.08146,-0.01833,0.08480,-0.00646,0.05803,0.09712,0.00762,-0.00440,0.07500,0.00517,-0.00053,0.00870,-0.00457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299551,2023-03-31 22:00:00,2833,267,983.0,0.08168,0.08136,0.05098,0.08213,-0.07219,0.07571,0.08407,0.01285,-0.00404,0.07844,-0.00940,-0.00124,0.02356,-0.01421
299552,2023-03-31 22:00:00,2842,268,262.0,0.04749,0.04863,-0.01230,0.04720,0.01530,0.04659,0.04495,-0.00054,-0.00184,0.04800,0.01892,0.00011,-0.00013,0.00249
299553,2023-03-31 22:00:00,2844,269,262.0,0.04748,0.04862,-0.01230,0.04720,0.01530,0.04658,0.04495,-0.00054,-0.00184,0.04800,0.01892,0.00011,-0.00013,0.00249
299554,2023-03-31 22:00:00,2850,0,908.0,-0.07457,-0.03530,0.00516,-0.08367,0.00179,0.00098,-0.09946,-0.00087,0.00050,-0.04448,-0.00075,0.00008,-0.00168,0.00058


In [59]:
df_ptdf_adj_reduced.to_excel("ptdf_z_obs.xlsx")